In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import os
import zipfile
import shutil

tree = ET.parse('C:\\Users\\1564427\\Documents\\Risk Heat Map_Master2017_Updated20180119.twb')
root = tree.getroot()

In [2]:
def find_branch(root,str_section):
   
    li=list(root)
    li=[i for i,x in enumerate(li) if x.tag==str_section]
        
    if len(li)==0:
        out=None
    else:
        out=li[0]
    
    return out

In [3]:
find_branch(twb_dir,'datasources')

NameError: name 'twb_dir' is not defined

In [3]:
#fields
##may not be needed as can use directly from worksheets
def parse_fields(str_dir):
    
    #parsing with elementtree
    tree = ET.parse(str_dir)
    root = tree.getroot()
    
    #container array
    arr_fields=[]
    
    #find section position
    section=find_branch(root,'datasources')
    
    ##looping through and parsing key fields
    for ds in root[section]:
        ds_name=ds.attrib.get('name')
        ds_caption=ds.attrib.get('caption')

        for col in ds.iter('column'):
            col_name=col.attrib.get('name')
            if col_name:
                col_name_cleaned=col_name[1:-1] # removing first and last characters, which are square brackets
            else:
                col_name_cleaned=col_name
            col_caption=col.attrib.get('caption')

            if col.attrib.get('param-domain-type'): #if it is a parameter
                col_type='parameter'
            else:
                col_type='calculation'

            #if len(col.getchildren())>0: 
            for calc in col.iter('calculation'):
                calc_class=calc.attrib.get('class')
                calc_formula=calc.attrib.get('formula')
                calc_column=calc.attrib.get('column')
                

                    
                arr_fields.append([#str_dir
                                   ds_name
                                   ,ds_caption
                                   ,col_name
                                   ,col_name_cleaned
                                   ,col_caption
                                   ,calc_class
                                   ,calc_formula
                                   ,calc_column
                                    ,col_type])
            
            #columns
        for meta in ds.iter('metadata-record'):
            if meta.attrib.get('class')=='column':
                for x in meta.iter('local-name'):
                    col_name=x.text
                for x in meta.iter('remote-alias'):
                    col_caption=x.text
                for x in meta.iter('remote-name'):
                    col_name_cleaned=x.text
                col_type='column'
                
                arr_fields.append([#str_dir
                   ds_name
                   ,ds_caption
                   ,col_name
                   ,col_name_cleaned
                   ,col_caption
                   ,None
                   ,None
                   ,None
                    ,col_type])


    df_fields=pd.DataFrame(data=arr_fields,columns=[#'Directory'
                                                    'DataSourceName'
                                                    ,'DataSourceCaption'
                                                    ,'ColumnName'
                                                    ,'ColumnNameCleaned'
                                                    ,'ColumnCaption'
                                                   ,'Class'
                                                   ,'Formula'
                                                   ,'CalculationColumn'
                                                    ,'ColumnType'])
    return df_fields


In [4]:
def parse_ws(str_dir):
            
    #parsing with elementtree
    tree = ET.parse(str_dir)
    root = tree.getroot()
    
    
    ##getting columns used in sheet
    arr_ws=[]
    
    #find section position
    section=find_branch(root,'worksheets')
    
    for ws in root[section]:
        ws_name=ws.attrib.get('name')
        #print(ws_name)
        for dsd in ws.iter('datasource-dependencies'):
            #print(dsd.attrib)
            dsd_name=dsd.attrib.get('datasource')
            for col in dsd.iter('column'):
                col_caption=col.attrib.get('caption')
                col_name=col.attrib.get('name') #raw name
                #col_name=col_name[1:-1] # removing first and last characters, which are square brackets
                col_role=col.attrib.get('role')
                #print(dsd_name,col_caption,col_name,col_role)
                arr_ws.append([#str_dir
                               ws_name
                               ,dsd_name
                               ,col_caption
                               ,col_name
                               ,col_role])

    df_ws=pd.DataFrame(data=arr_ws,columns=[#'Directory'
                                            'WorksheetName'
                                            ,'DataSourceName'
                                            ,'ColumnCaption'
                                            ,'ColumnName'
                                           ,'Role'])
    return df_ws


In [31]:
def parse_filter(str_dir):
    
    #parsing with elementtree
    tree = ET.parse(str_dir)
    root = tree.getroot()
    
    #getting filters
    arr_filter=[]
    arr_filter_member=[]
    
    sections=['worksheets','shared-views']
    sections=[s for s in sections if find_branch(root,s)!=None]
    
    for xml_section in sections:
        #find section position
        section=find_branch(root,xml_section)

        for ws in root[section]:
            ws_name=ws.attrib.get('name')
            #print(ws_name)
            for ft in ws.iter('filter'):
                ft_class=ft.attrib.get('class')
                ft_column=ft.attrib.get('column')
                ft_ds=ft_column.split('].')[0][1:] ## remove square bracket

                ft_column=ft_column.split('].')[1] #get the field name
                if ':' in ft_column: #split when colon is present
                    ft_column='[' + ft_column.split(':')[1] +']'
                else: # omit last character, for actions
                    ft_column=ft_column[1:-1]


                ft_group=ft.attrib.get('filter-group')
                arr_filter.append([#str_dir
                                   ws_name
                                   ,ft_class
                                   ,ft_ds
                                   ,ft_column
                                   ,ft_group])

                for gft in ft.iter('groupfilter'): #grouping layer of filter values
                    if gft.attrib.get('function') in ('member','range'):#when it is a member filter value
                        #print(gft.attrib)
                        gft_member=gft.attrib.get('member')
                        gft_from=gft.attrib.get('from')
                        gft_to=gft.attrib.get('to')
                        #print(gft_member,gft_from,gft_to)
                        arr_filter_member.append([#str_dir
                                                  ws_name
                                                  ,ft_class
                                                  ,ft_ds
                                                  ,ft_column
                                                  ,ft_group
                                                  ,gft_member
                                                  ,gft_from
                                                  ,gft_to])

    df_filter=pd.DataFrame(data=arr_filter,columns=[#'Directory'
                                                    'WorksheetName'
                                                    ,'FilterClass'
                                                    ,'DataSourceName'
                                                    ,'Column'
                                                    ,'FilterGroup'])

    df_filter_member=pd.DataFrame(data=arr_filter_member,columns=[#'Directory'
                                                                'WorksheetName'
                                                                ,'FilterClass'
                                                                ,'DataSourceName'
                                                                ,'Column'
                                                                ,'FilterGroup'
                                                               ,'Member'
                                                                 ,'From','To'])
    return df_filter,df_filter_member

#parse_filter(directory)[1]

In [6]:
def parse_dashboard(str_dir):
    
    #parsing with elementtree
    tree = ET.parse(str_dir)
    root = tree.getroot()
    
    #dashboards
    arr_db=[]
    
    #find section position
    section=find_branch(root,'dashboards')
    
    if section: # parse when dashboards are present
        for db in root[section]:
            db_name=db.attrib.get('name')
            for zone in db.iter('zone'):
                if zone.attrib.get('name') and not zone.attrib.get('type'):
                    arr_db.append([#str_dir
                                    db_name
                                   ,zone.attrib.get('name')])

    df_db=pd.DataFrame(data=arr_db,columns=[#'Directory'
                                            'DashboardName'
                                            ,'WorksheetName'])
    
    return df_db

#parse_dashboard(directory)

In [33]:
##creating shell dataframes
df_fields=pd.DataFrame(columns=['Directory'
                                ,'DataSourceName'
                                ,'DataSourceCaption'
                                ,'ColumnName'
                                ,'ColumnNameCleaned'
                                ,'ColumnCaption'
                               ,'Class'
                               ,'Formula'
                               ,'CalculationColumn'
                               ,'ColumnType'])
df_ws=pd.DataFrame(columns=['Directory'
                                ,'WorksheetName'
                                ,'DataSourceName'
                                ,'ColumnCaption'
                                ,'ColumnName'
                               ,'Role'])
df_filter=pd.DataFrame(columns=['Directory'
                                ,'WorksheetName'
                                ,'FilterClass'
                                ,'DataSourceName'
                                ,'Column'
                                ,'FilterGroup'])

df_filter_member=pd.DataFrame(columns=['Directory'
                                    ,'WorksheetName'
                                    ,'FilterClass'
                                    ,'DataSourceName'
                                    ,'Column'
                                    ,'FilterGroup'
                                   ,'Member'
                                     ,'From','To'])
df_db=pd.DataFrame(columns=['Directory'
                                ,'DashboardName'
                                ,'WorksheetName'])

In [34]:
folder_path="Y:\\3. Projects\\16. CRRC MI Refresh\\Tableau"

for file in os.listdir(folder_path):
    if file.endswith(".twb") or file.endswith(".twbx"):
        print(os.path.join(folder_path, file))
        file_dir=os.path.join(folder_path, file)
        
        if file.endswith(".twb"):
            twb_dir=os.path.join(folder_path, file)
        elif file.endswith(".twbx"): #extract packaged workbooks
            with zipfile.ZipFile(os.path.join(folder_path, file),"r") as zip_ref:
                zip_ref.extractall("targetdir")
                
            for file2 in os.listdir(os.path.join(os.getcwd(),'targetdir')):#in the extracted directory
                if file2.endswith(".twb"):
                    twb_dir=os.path.join(os.getcwd(),'targetdir',file2) #remove x from .twbx extension
        
        #fields
        temp=parse_fields(twb_dir)
        temp['Directory']=file_dir
        df_fields=pd.concat([df_fields,temp])
        
        #worksheets
        temp=parse_ws(twb_dir)
        temp['Directory']=file_dir       
        df_ws=pd.concat([df_ws,temp])
        
        #filters
        temp=parse_filter(twb_dir)[0]
        temp['Directory']=file_dir         
        df_filter=pd.concat([df_filter,temp])
        
        temp=parse_filter(twb_dir)[1]
        temp['Directory']=file_dir   
        df_filter_member=pd.concat([df_filter_member,temp])
        
        #dashboard
        temp=parse_dashboard(twb_dir)
        temp['Directory']=file_dir                              
        df_db=pd.concat([df_db,temp])
        
        if file.endswith(".twbx"): ##delete directory of extracted file
            shutil.rmtree('targetdir')

Y:\3. Projects\16. CRRC MI Refresh\Tableau\MI Dashboard v1.twb
Y:\3. Projects\16. CRRC MI Refresh\Tableau\MI Dashboard V10 with new data sources.twb
Y:\3. Projects\16. CRRC MI Refresh\Tableau\MI Dashboard V11.twb
Y:\3. Projects\16. CRRC MI Refresh\Tableau\MI Dashboard V2.twb
Y:\3. Projects\16. CRRC MI Refresh\Tableau\MI Dashboard V3.twb
Y:\3. Projects\16. CRRC MI Refresh\Tableau\MI Dashboard V4.twb
Y:\3. Projects\16. CRRC MI Refresh\Tableau\MI Dashboard V5.twb
Y:\3. Projects\16. CRRC MI Refresh\Tableau\MI Dashboard V6.twb
Y:\3. Projects\16. CRRC MI Refresh\Tableau\MI Dashboard V7.twb
Y:\3. Projects\16. CRRC MI Refresh\Tableau\MI Dashboard V8 CIB Final_Do not use.twb
Y:\3. Projects\16. CRRC MI Refresh\Tableau\MI Dashboard V8 CIB.twb
Y:\3. Projects\16. CRRC MI Refresh\Tableau\MI Dashboard V8.twb
Y:\3. Projects\16. CRRC MI Refresh\Tableau\MI Dashboard V9.twb
Y:\3. Projects\16. CRRC MI Refresh\Tableau\MI Dashboard.twb


In [35]:
#saving to excel
writer = pd.ExcelWriter('TWB.xlsx')
df_fields.to_excel(writer,'Fields',index=False)
df_ws.to_excel(writer,'WorkSheets',index=False)
df_filter.to_excel(writer,'Filters',index=False)
df_filter_member.to_excel(writer,'FilterMembers',index=False)
df_db.to_excel(writer,'Dashboards',index=False)
writer.save()

In [17]:
str_dir='Y:\\3. Projects\\16. CRRC MI Refresh\\Tableau\\MI Dashboard V10 with new data sources.twb'
tree = ET.parse(str_dir)
root = tree.getroot()

#getting filters
arr_filter=[]
arr_filter_member=[]


for xml_section in ['worksheets','shared-views']:
    #find section position
    section=find_branch(root,xml_section)

In [30]:
sections=['worksheets','shared-views']
[ s for s in sections if find_branch(root,s)!=None]

['worksheets']

In [29]:
(s for s in sections if find_branch(root,s)!=None)

SyntaxError: invalid syntax (<ipython-input-29-fc4e6ffc567c>, line 1)